<a href="https://colab.research.google.com/github/ayssag/7DaysOfCodePandas/blob/main/_7DaysOfCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando dados

In [ ]:
%%capture
!pip install pandas

In [6]:
repo = 'https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas.git'
!git clone {repo}

Cloning into '7_Days_of_Code_Alura-Python-Pandas'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 282 (delta 29), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (282/282), 82.88 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Updating files: 100% (33/33), done.


In [31]:
import os
import pandas as pd

path = '7_Days_of_Code_Alura-Python-Pandas/Dia_1-Importando_dados/Datasets'

files = os.listdir(f'{path}/dados_emprestimos')
emprestimos = pd.DataFrame()

for file in files:
  add_emprestimos = pd.read_csv(f'{path}/dados_emprestimos/{file}')
  emprestimos = pd.concat([emprestimos, add_emprestimos], ignore_index=True)


,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,1535182,E002010,NaN,2013/07/01 07:47:13.765000000,2013/08/06 20:28:21.000000000,2.012320e+09,ALUNO MÉDIO/TÉCNICO
1,1535183,2010049079,2013/07/16 17:25:10.615000000,2013/07/01 07:47:23.170000000,2013/07/18 09:50:26.000000000,2.009007e+09,ALUNO DE GRADUAÇÃO
2,1535184,2011013154,2013/07/16 21:03:55.928000000,2013/07/01 07:54:01.549000000,2013/07/30 08:39:22.000000000,2.011018e+09,ALUNO DE GRADUAÇÃO
3,1535185,2011001485,2013/07/16 21:03:55.802000000,2013/07/01 07:54:01.611000000,2013/07/17 08:56:58.000000000,2.011018e+09,ALUNO DE GRADUAÇÃO
4,1535186,2012038972,2013/07/16 17:51:37.835000000,2013/07/01 08:23:51.535000000,2013/07/31 13:51:11.000000000,2.011093e+09,ALUNO DE GRADUAÇÃO


In [32]:
files = os.listdir(path)
exemplares =  pd.read_parquet(f'{path}/dados_exemplares.parquet')

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


In [33]:
dataset = emprestimos.merge(exemplares, on='codigo_barras', how='left')

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,1535182,E002010,NaN,2013/07/01 07:47:13.765000000,2013/08/06 20:28:21.000000000,2.012320e+09,ALUNO MÉDIO/TÉCNICO,NaN,NaN,NaN,NaN,NaN,NaN
1,1535183,2010049079,2013/07/16 17:25:10.615000000,2013/07/01 07:47:23.170000000,2013/07/18 09:50:26.000000000,2.009007e+09,ALUNO DE GRADUAÇÃO,939119.0,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,668.0,169176.0
2,1535184,2011013154,2013/07/16 21:03:55.928000000,2013/07/01 07:54:01.549000000,2013/07/30 08:39:22.000000000,2.011018e+09,ALUNO DE GRADUAÇÃO,923040.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,660.0,166827.0
3,1535185,2011001485,2013/07/16 21:03:55.802000000,2013/07/01 07:54:01.611000000,2013/07/17 08:56:58.000000000,2.011018e+09,ALUNO DE GRADUAÇÃO,982496.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,310.0,179127.0
4,1535186,2012038972,2013/07/16 17:51:37.835000000,2013/07/01 08:23:51.535000000,2013/07/31 13:51:11.000000000,2.011093e+09,ALUNO DE GRADUAÇÃO,1025496.0,Acervo Circulante,Biblioteca Setorial da Faculdade de Ciências d...,REGULAR,610.0,166727.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261811,2782804,2015014347,NaN,2018/12/28 18:36:32.926000000,2019/02/01 14:42:41.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1187934.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,142.0,221485.0
2261812,2782805,2017015400,NaN,2018/12/28 18:36:32.955000000,2019/02/01 14:42:24.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1263731.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,552.0,242086.0
2261813,2782806,2008049611,2019/01/14 02:58:22.987000000,2018/12/28 18:58:54.794000000,2019/01/22 12:49:26.000000000,2.014047e+09,ALUNO DE GRADUAÇÃO,257460.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,604.0,110881.0
2261814,2782807,2015043579,2019/01/14 02:58:22.965000000,2018/12/28 18:58:54.830000000,2019/01/22 12:49:23.000000000,2.014047e+09,ALUNO DE GRADUAÇÃO,1209613.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,542.0,227220.0
